In [1]:
class StudentAnswer:
    ans = {}

answerKey = {'9':'Cloud Migration Factors : Security , Compliance, Cost, Business goals','10':'Time consuming, Modernized, migrated, time, money'} 
student1 = StudentAnswer()    

In [2]:
from sentence_transformers import SentenceTransformer
model_name='bert-base-nli-mean-tokens'
model=SentenceTransformer(model_name)


In [3]:
from fuzzywuzzy import fuzz

C:\Users\Shreinik Jain\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [5]:
from PIL import Image, ImageDraw
read_image = open("../images/image3.png", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(1)


Waiting for result...


In [6]:

from PIL import ImageOps
image = Image.open(read_image).convert('RGB')
draw = ImageDraw.Draw(image)




grayImage = ImageOps.grayscale(image)

colorHasher={}
counter=0
lineSum = 0
topbox = 0
bottombox = 0
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            lineSum += line.bounding_box[0]
            if counter==0:topbox=line.bounding_box[1]
            bottombox=line.bounding_box[5]
            counter+=1

marginLineLeft = 10
breakLoop = False
for top in range(10,int(topbox)):
    if(breakLoop):
        break
    for left in range(10,int(lineSum)//counter+10):
        if sum(image.getpixel((left,top)))<500:
            marginLineLeft=left
            breakLoop = True
            break
breakLoop = False
for top in range(int(bottombox),image.height-10):
    if(breakLoop):
        break
    for left in range(10,int(lineSum)//counter+10):
        if sum(image.getpixel((left,top)))<500:
            marginLineLeft=left
            breakLoop = True
            break
# margin
student1.ans = {}
draw.line((marginLineLeft,0,marginLineLeft,image.height-1), fill=(255,0,0))
qno = 0 
for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
        draw.rectangle(coordinates, outline=(0,150,0))



        if line.bounding_box[0]<marginLineLeft:
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline=(255,0,0))

            qno = line.text.split()[0].replace('.','')
            if(qno not in student1.ans):
                student1.ans[qno] = line.text[len(qno)+1:]
                continue
        if(qno!=0):
            student1.ans[qno] += " "
            student1.ans[qno] += line.text

image.show()
print(student1.ans)

{'9': ' Cloud Migration Factors : * Business goals * Security * Compliance * Cost and ROI * Disaster Recovery', '10': ' Workload Migration can be time consuming: Executing a cloud strategy can be time Consuming. And many applications have to be modernized to get them migrated which again Consumes time and money.'}


In [7]:
answerFactors = {'9':{'factor':'KEYWORD','ans':'Cloud Migration Factors : Security , Compliance, Cost, Business goals'},'10':{'factor':'KEYWORD','ans':'Leadership, skill, important'}} 
answerKey = {'9':'Cloud Migration Factors : Security , Compliance, Cost, Business goals','10':'Leadership, skill, important'}

In [8]:
answerKey_vectors = model.encode(list(answerKey.values()))
student1_vectors = model.encode(list(student1.ans.values()))

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
result={}
for i,j in enumerate(answerKey_vectors):
    temp = cosine_similarity([j],[student1_vectors[i]])[0][0]
    sr=len(list(student1.ans.values())[i])
    ar=len(list(answerKey.values())[i])
    finr=sr/ar if sr<ar else 1
    fuzzscore=fuzz.token_set_ratio(student1.ans[list(answerKey.keys())[i]],answerKey[list(answerKey.keys())[i]])
    if answerFactors[list(answerKey.keys())[i]]['factor']=='KEYWORD':
        result[list(answerKey.keys())[i]] = fuzzscore/100
    if answerFactors[list(answerKey.keys())[i]]['factor']=='SIMILARITY':
        result[list(answerKey.keys())[i]] = temp*finr/2+fuzzscore/200
result


{'9': 1.0, '10': 0.16}

In [10]:
import math
for k in answerKey.keys():
    if(k not in student1.ans):
        print("Question "+k+" not answered")
    else:
        print("Question "+k+" answered")
        print("Marks alloted : "+ str(math.ceil(result[k]*4)/2))

Question 9 answered
Marks alloted : 2.0
Question 10 answered
Marks alloted : 0.5
